## Importing Necessary Library

In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from bertopic import BERTopic
from nltk.stem import WordNetLemmatizer
from nltk import ngrams
import gensim.corpora as corpora
import tensorflow_hub as hub
import time
import gensim
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
from keybert import KeyBERT
import tensorflow as tf

#### Loading training dataset

In [2]:
train_data = pd.read_csv('train_data.csv')

In [3]:
# ==== printng head of data
train_data.head(10)

,path,transcription,action,object,location
0,wavs/speakers/xRQE5VD7rRHVdyvM/7372ca00-45c4-1...,Turn on the kitchen lights,activate,lights,kitchen
1,wavs/speakers/R3mexpM2YAtdPbL7/dae28110-44fe-1...,Turn up the temperature,increase,heat,none
2,wavs/speakers/ZebMRl5Z7dhrPKRD/b55dcfd0-455d-1...,OK now switch the main language to Chinese,change language,Chinese,none
3,wavs/speakers/ppzZqYxGkESMdA5Az/61c54a20-4476-...,Turn down the bathroom temperature,decrease,heat,washroom
4,wavs/speakers/zaEBPeMY4NUbDnZy/8ef57ec0-44df-1...,Change the language,change language,none,none
5,wavs/speakers/W4XOzzNEbrtZz4dW/8a1fedc0-44d2-1...,Turn sound down,decrease,volume,none
6,wavs/speakers/8e5qRjN7dGuovkRY/92366b20-459e-1...,Decrease the heating in the washroom,decrease,heat,washroom
7,wavs/speakers/W4XOzzNEbrtZz4dW/d40180c0-44d2-1...,"It’s too loud, turn it down",decrease,volume,none
8,wavs/speakers/g2dnA9Wpvzi2WAmZ/694d7090-454a-1...,Change language,change language,none,none
9,wavs/speakers/5BEzPgPKe8taG9OB/b9b56e80-457b-1...,Turn the lights on in the kitchen,activate,lights,kitchen


In [4]:
train_data = train_data.drop(['path'], axis = 1)  # dropping path from dataset

In [5]:
# ==== merging action, object and location column
l = []
def merge(data):
    for i in range(len(data)):
        l.append(data["action"] + " " + data["object"] + " " + data["location"])
    return l


In [6]:
# ==== calling merge function
merge(train_data)

[0             activate lights kitchen
 1                  increase heat none
 2        change language Chinese none
 3              decrease heat washroom
 4           change language none none
                      ...             
 11561           increase heat kitchen
 11562              increase heat none
 11563                bring shoes none
 11564            decrease volume none
 11565       deactivate lights kitchen
 Length: 11566, dtype: object,
 0             activate lights kitchen
 1                  increase heat none
 2        change language Chinese none
 3              decrease heat washroom
 4           change language none none
                      ...             
 11561           increase heat kitchen
 11562              increase heat none
 11563                bring shoes none
 11564            decrease volume none
 11565       deactivate lights kitchen
 Length: 11566, dtype: object,
 0             activate lights kitchen
 1                  increase heat none
 2

In [7]:
train_data['merge'] = l[0]

In [8]:
len(train_data['merge'].unique()) # getting number of unique values in merge column

31

In [9]:
train_data.groupby('merge').describe() # showing description of merge column

transcription         \
                                     count unique   
merge                                               
activate lamp none                     187      4   
activate lights bedroom                275      6   
activate lights kitchen                386      8   
activate lights none                   248      5   
activate lights washroom               446     10   
activate music none                    359      7   
bring juice none                       243      5   
bring newspaper none                   274      6   
bring shoes none                       266      6   
bring socks none                       265      6   
change language Chinese none           229      5   
change language English none           196      4   
change language German none            161      4   
change language Korean none            169      4   
change language none none              512     11   
deactivate lamp none                   208      4   
deactivate lights bedroom              283      6   
deactivate lights kitchen              264      6   
deactivate lights none                 242      5   
deactivate lights washroom             219      5   
deactivate music none                  321      7   
decrease heat bedroom                  311      7   
decrease heat kitchen                  308      7   
decrease heat none                     510     11   
decrease heat washroom                 663     14   
decrease volume none                  1060     22   
increase heat bedroom                  316      7   
increase heat kitchen                  315      7   
increase heat none                     576     11   
increase heat washroom                 630     14   
increase volume none                  1124     24   

                                                                                 \
                                                                            top   
merge                                                                             
activate lamp none                                           Switch on the lamp   
activate lights bedroom                            Switch the bedroom lights on   
activate lights kitchen                                       Kitchen lights on   
activate lights none                                         Turn the lights on   
activate lights washroom                            Turn the bathroom lights on   
activate music none                                             Start the music   
bring juice none                                              Get me some juice   
bring newspaper none                                     Bring me the newspaper   
bring shoes none                                             Go get me my shoes   
bring socks none                                                 Fetch my socks   
change language Chinese none  I need to practice my Chinese. Switch the lang...   
change language English none  I need to practice my English. Switch the lang...   
change language German none                   Set my phone's language to German   
change language Korean none           OK now switch the main language to Korean   
change language none none                                   Change the language   
deactivate lamp none                                        Switch off the lamp   
deactivate lights bedroom                         Switch the bedroom lights off   
deactivate lights kitchen                                    Kitchen lights off   
deactivate lights none                                    Switch off the lights   
deactivate lights washroom                                  Washroom lights off   
deactivate music none                                        Turn off the music   
decrease heat bedroom                   Decrease the temperature in the bedroom   
decrease heat kitchen                   Decrease the temperature in the kitchen   
decrease heat none                                           Turn do

In [10]:
# label encoding the merge column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
r = le.fit_transform(train_data['merge'])

In [11]:
train_data['merge_vector'] = r

In [12]:
train_data.head(10)   # prining head of the updated data

,transcription,action,object,location,merge,merge_vector
0,Turn on the kitchen lights,activate,lights,kitchen,activate lights kitchen,2
1,Turn up the temperature,increase,heat,none,increase heat none,28
2,OK now switch the main language to Chinese,change language,Chinese,none,change language Chinese none,10
3,Turn down the bathroom temperature,decrease,heat,washroom,decrease heat washroom,24
4,Change the language,change language,none,none,change language none none,14
5,Turn sound down,decrease,volume,none,decrease volume none,25
6,Decrease the heating in the washroom,decrease,heat,washroom,decrease heat washroom,24
7,"It’s too loud, turn it down",decrease,volume,none,decrease volume none,25
8,Change language,change language,none,none,change language none none,14
9,Turn the lights on in the kitchen,activate,lights,kitchen,activate lights kitchen,2


In [13]:
train_data['location'].unique()

array(['kitchen', 'none', 'washroom', 'bedroom'], dtype=object)

In [14]:
train_data.shape          # Shape of train data

(11566, 6)

In [15]:
train_data.columns           # columns present in train data

Index(['transcription', 'action', 'object', 'location', 'merge',
       'merge_vector'],
      dtype='object')

In [16]:
train_data.isnull().sum()        # checking fot the null value 

transcription    0
action           0
object           0
location         0
merge            0
merge_vector     0
dtype: int64

In [17]:
# === Spliting the dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(train_data['transcription'], train_data['merge_vector'], stratify= train_data['merge_vector'])

In [18]:
X_train.head(5)   # showing head of train data

7892              Lights on in the kitchen
4496                      Set the language
5504           Turn the washroom lights on
6817    Turn the heat down in the washroom
5610                               Lamp on
Name: transcription, dtype: object

In [19]:
import tensorflow_text as text

In [20]:
# calling bert preprocessor and bert encoder 

bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

INFO:absl:Using C:\Users\soni1\AppData\Local\Temp\tfhub_modules to cache modules.


In [21]:
# ==== sample test 
def gets(s):
    pt = bert_preprocess(s)
    return bert_encoder(pt)['pooled_output']

gets([
    "Turn on the kitchen lights",
    "Decrease the heating in the washroom"])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8874407 , -0.1622439 , -0.16199206, ...,  0.068757  ,
        -0.6292149 ,  0.9247728 ],
       [-0.92010224, -0.40742946, -0.9761191 , ..., -0.9196166 ,
        -0.6752491 ,  0.8806683 ]], dtype=float32)>

In [22]:
## creating layers for model

# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(8, activation='relu')(l)
l = tf.keras.layers.Dense(6, activation='relu')(l)
l = tf.keras.layers.Dense(4, activation='relu')(l)
l = tf.keras.layers.Dense(2, activation='relu')(l)
l = tf.keras.layers.Dense(1, activation='relu', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])



In [23]:
# showing model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           text[0][0]                       
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'encoder_outputs':  109482241   keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [24]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
start = time.time()
model.fit(X_train,
          Y_train, 
          epochs=10)
end = time.time()
total_time = round(end-start,2)
print("\n\nModel took {} seconds to train".format(total_time))

Epoch 1/10
272/272 [==============================] - 1379s 5s/step - loss: nan - accuracy: 0.0161
Epoch 2/10
272/272 [==============================] - 1373s 5s/step - loss: nan - accuracy: 0.0161
Epoch 3/10
272/272 [==============================] - 1375s 5s/step - loss: nan - accuracy: 0.0161
Epoch 4/10
272/272 [==============================] - 1493s 5s/step - loss: nan - accuracy: 0.0161
Epoch 5/10
272/272 [==============================] - 1395s 5s/step - loss: nan - accuracy: 0.0161
Epoch 6/10
272/272 [==============================] - 1380s 5s/step - loss: nan - accuracy: 0.0161
Epoch 7/10
272/272 [==============================] - 1378s 5s/step - loss: nan - accuracy: 0.0161
Epoch 8/10
257/272 [===========================>..] - ETA: 1:19 - loss: nan - accuracy: 0.0161

### Model training will take more 3 - 4 hrs time it's not yet completed, my system is not having graphics card thta's why it's is taking more and more time. Hence, I'm submitting it beacuse of deadline. Please consider this.